<a href="https://colab.research.google.com/github/miguelvelascop/PFG_Miguel_Velasco_Puig/blob/main/Preprocesado_Dataset_DemonicSalmon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan librerías para manejar vectores y matrices (numpy) y manejar tablas (panda) más facilmente, y os, que permite moverse por el sistema de archivos más facilmente(en este caso por GoogleDrive)

In [ ]:
import numpy as np
import pandas as pd
import os

Función que transforma una magnitud vectorial en una magnitud escalar

In [ ]:
import math
def vectorial_escalar(x, y, z):
  escalar = np.sqrt(x**2 + y**2 + z**2)
  return escalar

Se importan librerías que me permiten cargar archivos desde local y desde drive

In [ ]:
from google.colab import files, drive

In [ ]:
df = pd.DataFrame()
#Cargamos dos listas con todos los archivos csv del dataset (
ruta_aceleromentro='/content/drive/MyDrive/TFG/datasets/DemonicSalmon/Passive_data/Accelerometer'
ruta_llamadas='/content/drive/MyDrive/TFG/datasets/DemonicSalmon/Passive_data/Calls'
ruta_SMS='/content/drive/MyDrive/TFG/datasets/DemonicSalmon/Passive_data/SMS texts'
ruta_datosPers='/content/drive/MyDrive/TFG/datasets/DemonicSalmon/Lab_measures/Lab_measures.csv'
archivos_acel = os.listdir(ruta_aceleromentro)
archivos_call = os.listdir(ruta_llamadas)
archivos_SMS = os.listdir(ruta_SMS)

  Selección de etiquetas, ejecutar el primer bloque para mantener valores originales en las etiquetas y el segundo bloque para convertirlo en una etiqueta binaria (0 no depresión, 1 depresión)
  

In [ ]:
etiquetaBinaria = True

In [ ]:
etiquetaBinaria = False

Primero se va leyendo el csv de cada voluntario, se obtiene el valor escalar del acelerometro, y lo se agrupa por dias para obtener un datata frame con
el día y la actividad asociada

In [ ]:
datosPers = pd.read_csv(ruta_datosPers,  sep=',',  comment='#') #En esta línea cargamos los datos sobre el nivel de depresión de los voluntarios
datosPers['PID'].to_string()

ini = True
for i in archivos_acel:
  person = i[0:18]
  pNumero = i[14:18]
  aux=pd.read_csv(ruta_aceleromentro+'/'+i,  sep=',',  comment='#')
  time = aux['Timestamp'].to_numpy()
  acelSeg = (vectorial_escalar(aux['X'],aux['Y'],aux['Z']))
  matriz = np.column_stack((time, acelSeg))
  data = pd.DataFrame(matriz, columns = ['Timestamp','Activity'])
  data['Timestamp'] = data['Timestamp'].str.slice(0,11)
  #En las siguientes líneas se añaden los datos acumulados de la actividad de cada voluntario y métricas asociadas a esta actividad
  dataGrup = data.groupby('Timestamp').agg({'Activity': 'sum'}).reset_index()
  dataGrup['Media'] = data.groupby('Timestamp')['Activity'].mean().to_numpy()
  dataGrup['Desviacion'] = data.groupby('Timestamp')['Activity'].std().to_numpy()
  dataGrup['Minimo'] = data.groupby('Timestamp')['Activity'].min().to_numpy()
  dataGrup['Maximo'] = data.groupby('Timestamp')['Activity'].max().to_numpy()
  #A continuación se añadan los campos de Llamadas y SMS vacíos
  dataGrup['LlamadasSal'] = 0
  dataGrup['LlamadasEntr'] = 0
  dataGrup['LlamadasPer'] = 0
  dataGrup['SMS'] = 0
  #Añadir etiquetas en función de los datos de depresión que tenemos de los voluntarios, si no tienen datos de depresión se descarta
  incluir=False
  for j in range(datosPers.shape[0]):
    if(int(pNumero)==datosPers.loc[j,'PID']):
      if(np.isnan(datosPers.loc[j,'DASS.D'])):
        dataGrup['Etiqueta']=-1
      else:
        if (etiquetaBinaria):
          if(datosPers.loc[j,'DASS.D']>=5): dataGrup['Etiqueta']=1
          if(datosPers.loc[j,'DASS.D']<5): dataGrup['Etiqueta']=0
        else:
           dataGrup['Etiqueta'] = int(datosPers.loc[j,'DASS.D'])

        incluir=True
  #Ahora se comprueba por cada día las llamadas entrantes, salientes y perdidas registradas y se añaden al dataframe
  for j in archivos_call:
    if (person in j):
      call = pd.read_csv(ruta_llamadas+'/'+j,  sep=',',  comment='#')
      for k in range(dataGrup.shape[0]):
        for l in range(call.shape[0]):
          if (dataGrup.loc[k, 'Timestamp'] in call.loc[l, 'Timestamp']):
            if (call.loc[l, 'State'] == 'idle'): dataGrup.loc[k,'LlamadasPer' ]+=1
            if (call.loc[l, 'State'] == 'Ongoing'): dataGrup.loc[k,'LlamadasSal' ]+=1
            if (call.loc[l, 'State'] == 'Incoming'): dataGrup.loc[k,'LlamadasEntr' ]+=1
#Se hace lo mismo con los SMS
  for j in archivos_SMS:
    if (person in j):
      sms = pd.read_csv(ruta_SMS+'/'+j,  sep=',',  comment='#')
      for k in range(dataGrup.shape[0]):
        for l in range(sms.shape[0]):
          if (dataGrup.loc[k, 'Timestamp'] in sms.loc[l, 'Timestamp']):
            dataGrup.loc[k,'SMS' ]+=1
  #Ahora se van uniendo los datos de cada voluntario en un solo dataframe
  if (incluir):
    if(ini):
      dataFin = dataGrup
      ini=False
    else: dataFin = pd.concat([dataFin, dataGrup])
  print(dataFin)

1088
Filtro
True Si
18
Incluyendo
      Timestamp      Activity     Media  Desviacion    Minimo     Maximo  \
0   Fri Mar 18   16330.581271  0.191608    0.536298  0.000649  21.837002   
1   Fri Mar 25    12314.34793  0.146883    0.475684  0.000584  12.831381   
2   Mon Mar 14    6215.768551  0.249099    0.645410  0.002279  20.595348   
3   Mon Mar 21   10458.030214  0.123154    0.435505  0.000778  21.147489   
4   Mon Mar 28   14068.051566  0.165114    0.546772  0.000837  26.128885   
5   Sat Mar 19   10557.623928  0.123833    0.451633  0.000582  13.322009   
6   Sat Mar 26   12512.088187  0.146484    0.467435  0.000439  14.361504   
7   Sun Mar 20   10399.723019  0.122876    0.410055  0.001158   16.32957   
8   Sun Mar 27   16363.367464  0.191465    0.588255  0.000596  30.087737   
9   Thu Mar 17   12836.519342  0.150839    0.513371  0.000841  17.675985   
10  Thu Mar 24    12635.96786  0.148486    0.566735  0.000674  17.478179   
11  Thu Mar 31    5399.370737  0.158772    0.524194  0

Por último, se importa el dataframe a un csv y se descarga

In [ ]:
#Con index=False le quitamos los índices
dataFin = dataFin.drop('Timestamp', axis=1) #Quitamos los timestamp
dataFin = dataFin.dropna() #Quitamos filas con campos vacíos
if(etiquetaBinaria):
  dataFin.to_csv('Dataset_DemonicSalmon_procesado_etiqBinarias.csv', index=False)
  files.download('Dataset_DemonicSalmon_procesado_etiqBinarias.csv')
else:
  dataFin.to_csv('Dataset_DemonicSalmon_procesado_etiqOriginales.csv', index=False)
  files.download('Dataset_DemonicSalmon_procesado_etiqOriginales.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>